# CSE 4303 Computer Graphics

## Honors Project

### Swangya Saurav

## Assignment 1

**Task : ** Add a "Load" command (button) to Assignment_00 such that the program will be able to read the data from an input file and display the data on the viewport.

********
Importing important libraries

In [1]:
import ipywidgets as widgets
from IPython.display import display
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import Rectangle
import numpy as np


In [2]:
#Global Variables
global lines
lines = []

In [3]:
def f():    
    fig1 = plt.figure()
    ax1 = fig1.add_subplot(111, aspect='equal')
    ax1.add_patch(
    patches.Rectangle(
        (10, 10),   # (x,y)
        50,          # width
        50,          # height
        fill=False
        )
    )
    ax1.set_ylim([0, 100])
    ax1.set_xlim([0, 100])
    plt.show()

In [4]:
def getFiledata(filename):
    a =[]
    with open(filename) as textFile:
        a = [line.split() for line in textFile]
    list = [x for x in a if x != []]
    lines = list
    return list

w = interact_manual(getFiledata, filename = widgets.RadioButtons(options=["pyramid.txt", "cow.txt", "teapot.txt"]))

In [10]:
def create_vertex_list(data):
    vertex_list = []
    for element in data:
        if element[0] == 'v':
            vertex_list.append([float(element[1]), float(element[2]), float(element[3])])
    return vertex_list

In [19]:
def create_edge_list(data):
    edge_list = []
    for element in data:
        if element[0] == 'f':
            edge_list.append([float(element[1]), float(element[2]), float(element[3])])
    return edge_list

In [21]:
def get_viewport(data):
    view_dimension = []
    for element in data:
        if element[0] == 's':
            dimension = element
            break
    view_dimension = [float(dimension[1]),float(dimension[2]), float(dimension[3]), float(dimension[4])]

In [22]:
def get_window(data):
    for element in data:
        if element[0] == 'w':
            dimension = element
            break
    window_dimension = [float(dimension[1]),float(dimension[2]), float(dimension[3]), float(dimension[4])]

In [24]:
def translateViewport(xmin, ymin, xmax, ymax, width, height):
    a = width
    b = height
    x = float(xmin)*float(a);
    y = float(ymin)*float(b);
    width = (float(xmax) - float(xmin))*float(a)
    height = (float(ymax) - float(ymin))*float(b)
    u = x+width
    v = y+height
    dimensions = [x, y, u, v]
    self.viewFrameDimensions = dimensions
    return dimensions

In [35]:
def translate_points(vertex_list):
    translated_points = []
    for element in vertex_list:
        temp = translate_coordinate(element[0], element[1])
        translated_points.append(temp)
    return(translated_points)

In [30]:
def translate_coordinate(self, pwx, pwy):
    a = self.window_dimension
    b = self.view_dimension

    pwx = float(pwx)
    pwy = float(pwy)

    xwmin = float(a[0])
    xwmax = float(a[2])
    ywmin = float(a[1])
    ywmax = float(a[3])
    nxvmin = float(b[0])
    nxvmax = float(b[2])
    nyvmin = float(b[1])
    nyvmax = float(b[3])
    screen_width = float(self.width)
    screen_height = float(self.height)

    xvmin = float(nxvmin * screen_width)
    xvmax = float(nxvmax * screen_width)
    yvmin = float(nyvmin * screen_height)
    yvmax = float(nyvmax * screen_height)

    sx = (xvmax - xvmin) / (xwmax - xwmin)
    psx = xvmin + float(sx * (pwx - xwmin))
    sy = (yvmax - yvmin) / (ywmax - ywmin)
    psy = yvmin + float(sy * (ywmax - pwy))

    return [psx, psy]

In [36]:
data = getFiledata("pyramid.txt")
vertex = create_vertex_list(data)
edge = create_edge_list(data)
view = get_viewport(data)
window = get_window(data)
vertex = translateViewport(vertex)
print(edge)

TypeError: translateViewport() missing 6 required positional arguments: 'xmin', 'ymin', 'xmax', 'ymax', 'width', and 'height'